In [2]:
import pandas as pd
import sqlite3
from datetime import datetime

# Importing 3 Dataframes

In [3]:
df_parcel = pd.read_csv('../Data/raw/EXTR_Parcel.csv', encoding = "ISO-8859-1")

In [4]:
df_res = pd.read_csv('../Data/raw/EXTR_ResBldg.csv',low_memory=False)

In [5]:
df_real = pd.read_csv('../Data/raw/EXTR_RPSale.csv', encoding = "ISO-8859-1", low_memory=False)

# Filtering for single family homes only

In [6]:
df_real['DocumentDate'] = pd.to_datetime(df_real['DocumentDate'], format="%m/%d/%Y")
df_real['year'] = pd.DatetimeIndex(df_real['DocumentDate']).year

In [7]:
#set property type to 2,3,11 which are single family units
#set to principal use to 6 which is RESIDENTIAL   
#set year to 2019 for most recent data
# set propertyclass to 8 and 7 which includes only houses
df_real = df_real.loc[(df_real.PropertyType.isin([2,3,11])) & 
                           (df_real.SalePrice > 0) & 
                           (df_real.PrincipalUse == 6) & 
                           (df_real.year == 2019) &
                           (df_real.PropertyClass.isin([8,7]))]

# Creating a UniqueID accross all dataframes to combine later

In [8]:
df_list = [df_parcel, df_res, df_real]

In [9]:
for i in df_list:
    i.Minor = i.Minor.apply(lambda x: str(x).zfill(4))
    i.Major = i.Major.apply(lambda x: str(x).zfill(6))
    i['UniqueID'] = pd.to_numeric(i['Major'] + i['Minor']).astype(int)

In [26]:
df_res.UniqueID.dtype

dtype('int64')

# Removing duplicates

In [22]:
#dropping duplicated values of unique id in and keeping most recent sale
df_real = df_real.sort_values('DocumentDate').drop_duplicates('UniqueID', keep='last')

In [23]:
df_res = df_res.drop_duplicates(subset = 'UniqueID', keep = 'first')

# Making UniqueID column the index for all the dataframes

In [30]:
df_res.set_index(keys = 'UniqueID', drop=True, inplace=True)
df_parcel.set_index(keys = 'UniqueID', drop=True, inplace=True)
df_real.set_index(keys = 'UniqueID', drop=True, inplace=True)

# Concatenating the Dataframes 

In [32]:
test_df = pd.concat([df_real, df_parcel.reindex(df_real.index)], axis=1)

In [34]:
df_combined = pd.concat([test_df, df_res.reindex(test_df.index)], axis=1)

In [35]:
# Removes duplicated columns
df_combined = df_combined.loc[:,~df_combined.columns.duplicated()]

In [36]:
df_final = df_combined[['DocumentDate','SalePrice', 'PropertyType', 'Area', 'SqFtLot', 'LakeWashington',
 'LakeSammamish','SmallLakeRiverCreek','WfntLocation','WfntFootage','WfntBank','WfntPoorQuality','WfntRestrictedAccess','TidelandShoreland', 'TrafficNoise',
 'PowerLines','OtherNuisances', 'SqFtTotLiving','SqFtDeck','SqFtOpenPorch', 'Condition' ]]

In [37]:
df_final.to_json('../Data/combined.json')